In [239]:
from simple_salesforce import Salesforce
import os, json, random
import pandas as pd

In [129]:
SF = Salesforce("iet@detroitmi.gov", os.environ['SF_PASS'], os.environ['SF_TOKEN'])

In [254]:
## helper functions
def display_query(query):
    print(json.dumps(SF.query(query), indent=1))
    return SF.query_all(query)

def display_obj(obj):
    print(json.dumps(obj, indent=1))

def get_count():
    return display_query("select count(Id) from Account")

def process_record(record, fieldmap):
    data = {}
    for k, v in fieldmap.items():
        try:
            data[k] = record[v]
        except: 
            print("no match on field: {}".format(v))
    return data

acct_fieldmap = {
    'Parcel ID': 'Parcel_ID__c',
    'Address': 'Name',
    'Current Status':	"Current_Status__c",
    'Current Status Date':	"Current_Status_Date__c",
    'Property Ownership':	"Property_Ownership__c",
    'Pipeline':	"Pipeline__c",
    'Program':	"Program__c",
    'Acquisition Amount':	"Acquisition_Amount__c",
    'Acquisition Date':	"Acquisition_Date__c",
    'Acquisition Method':	"Acquisition_Method__c",
    'Foreclosure Year':	"Foreclosure_Year__c",
    'BTF Recommendation':	"BTF_Recommendation__c",
    'Occupancy Model Probability':	"Occupancy_Model_Probability_Pct__c",
    'Occupancy Model Last Update':	"Occupancy_Model_Last_Update__c",
    'Photo URL':	"Photo_URL__c",
    'Case GTJ Photo Link':	"Case_GTJ_Photo_Link__c"
    }
status_whitelist = [
    'For Sale',
    'Vacant Land',
    'Intake Review',
#     'Closed (No Compliance)',
    'Sales Review',
    'Demo Review',
    'Demolished',
    'Demo Pipeline',
#     'Occupied',
#     'Not Postered',
    'Demo Awaiting Funding',
    'Dismissed',
#     'Institutional Partner Coordination',
#     'PDD Quarantine',
    'Compliance (Sales)',
    'Demo Contracted',
    'Compliance Achieved (NAP)',
    'Sold',
    'Approved for Postering',
    'Cancelled',
#     'Drug House Identified',
    'Compliance Achieved (Sales)',
    'Sales Pipeline',
#     'DHU Letter Sent',
#     'Release of Interest',
#     'Judgment to DLBA',
#     'Sales Hold',
#     'Lease/ROE/TLA',
    'Compliance (NAP)',
#     'Tax Problem Identified',
#     'Corrective Transfer',
#     'WCT Quarantine',
#     'Warning Letter',
    'Compliance (Buy Back)',
    'WCT Unsold',
#     'DJ Quarantine',
#     'Property Rejected',
#     'Warehouse',
#     'Compliance (CP)',
#     'Donation Review',
#     'NPOC Isolation - Do not move!',
#     'Potential Reconveyance',
#     'Negotiating Agreement',
#     'NSP Quarantine',
#     'Approved For Complaint',
#     'Court Action Recommended',
#     'Second Raid',
#     'Reconveyance In Process',
#     'Consent Judgment',
#     'Case Filed',
#     'DJ Set Aside',
#     'Court Activity',
#     'Prep Dismissal',
#     'Title Transfer In Progress',
#     'Service In Process',
    'Under Construction',
#     'Title Problem Identified',
    'We Buy It',
#     'Construction Complete',
#     'Postered (Notice Sent)',
    'We Buy It Pilot',
#     'Quiet Title in Process',
#     'Prep Default',
#     'Canceled',
]
pipeline_whitelist = [
    'Inventory',
    'Side Lot',
    'HHF Demo',
    'Own It Now',
    'Non-HHF Demo',
    'Economic Development',
    'NAP',
    'Non-DLBA Owned Demo',
#     'Drug House',
    'Auction',
    'Community Partner',
    'Buy Back',
    'DBA Commercial Demo',
    'Prospective DLBA Property',
    'DLBA Owned Commercial Demo',
    'Rehabbed and Ready'
]
def scrub_account_values(record):
    if record['Current Status'] not in status_whitelist:
        record['Current Status'] = None
    if record['Pipeline'] not in pipeline_whitelist:
        record['Pipeline'] = None
    return record

def get_acct(id):
    data = SF.Account.get(id)
    filtered = process_record(data, acct_fieldmap)
    return data, scrub_account_values(filtered)

case_fieldmap = {
    'NTP Batch': "DEMO_NTP_Batch__c",
    'NTP Date': "DEMO_NTP_Date__c",
    'Asbestos Survey RFP Group': "DEMO_ASB_Survey_RFP__c",
    'Util Cut Status': "DEMO_UTIL_Cut_Status__c",
    'Planned Knock Down Date': "DEMO_Planned_Knock_Down_Date__c",
    'Knock Down Date': "DEMO_Knock_Down_Date__c",
    'Demo RFP Group': "Demo_RFP_Group__c",
    'Demo Contractor': "Demo_Contractor_TEXT_ONLY__c",
    'Demo Contractor Proceed Date': "Demo_Contractor_Proceed_Date__c",
    'Bid Price Contractor Amount': "DEMO_Total_Demo_Contractor_Amt_Formula__c",
    'Grant Program': "Grant_Program__c",
    'Sold - Auction': "Sold_Auction__c",
    'Sold - Side Lot': "Sold_Side_Lot__c",
    'Sold - Direct Sale': "Sold_Direct_Sale__c",
    'Sold Date': "Sold_Date__c"
}
def get_case(id):
    data = SF.Case.get(id)
    filtered = process_record(data, case_fieldmap)
    return data, filtered

def process_parcel(id):
    acct = get_acct(id)
    case = get_case(acct[0]['Related_Property_Case__c'])
    # merge two dictionaries
    return {**acct[1], **case[1]}

def get_sample():
    rows = []
    all_ids = SF.query_all("Select Id FROM Account WHERE Property_Ownership__c in ('DLBA Owned', 'Formerly DLBA Owned') LIMIT 5000 OFFSET {}".format(random.randrange(1,2000)))
    ids = [r["Id"] for r in all_ids['records']]
    for i in ids:
        rows.append(process_parcel(i))
    return pd.DataFrame(rows)

In [255]:
dlba_df = get_sample()

In [258]:
dlba_df.to_csv('dlba_sample.csv')